In [1]:
import pandas as pd
import numpy as np
from sklearn import tree, metrics
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
from io import StringIO  
from IPython.display import Image  
!pip install pydotplus 
import pydotplus 
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, f_regression
import datetime


# Home/Away Winner Prediction Model

In [2]:
df2 = pd.read_excel(r'/Users/markclampitt/Documents/Springboard/Capstone2/Data\new_df.xlsx')

In [3]:
df2.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,3.0,1.0,3,1,0.0,5.0,1.0,5.0,NaN,NaN
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,1.0,3.0,3,1,5.0,0.0,5.0,0.0,24.0,41.0
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,11.0,99.0,99,15,16.0,11.0,21.0,11.0,24.0,27.0
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,1.0,99.0,99,3,11.0,5.0,16.0,5.0,24.0,22.0
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99.0,99.0,99,99,25.0,13.0,35.0,18.0,45.0,21.0


In [4]:
#y variable created below ('H/A Winner') which is whether the home or away team won. 

In [5]:
df2['H/A Winner'] = df2.apply(lambda x: 'Home' if x['Home Score_x_x'] > x['Away Score_x_x'] else ('Away' if x['Away Score_x_x'] > x['Home Score_x_x'] else ""),axis=1)

In [6]:
# Removed actual scores and only kept data that would be available at the start of the game
# After testing the final rf model, I noticed that it perfomed better without the conference/season records as a features

In [7]:


df2 = df2.drop('Home Score_x_x',axis =1)
df2 = df2.drop('Away Score_x_x',axis =1)
df2 = df2.drop('Rolling 5 Past H&A Season T Losses_Away_Team',axis =1)
df2 = df2.drop('Rolling 5 Past H&A Season T Losses_Home_Team',axis =1)
df2 = df2.drop('Rolling 5 Past H&A Conf T Losses_Away_Team',axis =1)
df2 = df2.drop('Rolling 5 Past H&A Conf T Losses_Home_Team',axis =1)


In [8]:
Prediction = df2[df2['H/A Winner'] == ""] #This is the championship game (Georgia ('Home')/Alabama ('Away'))
Prediction 

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Total Yards Avg_Away_Team,Rolling 5 Past H&A Yards per Pass_Away_Team,Rolling 5 Past H&A Yards per Pass Avg_Away_Team,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,H/A Winner
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,495.4,46.5,9.3,-87.5,-17.5,3.0,1.0,3,1,


In [9]:
df2.shape

(977, 32)

In [10]:
NoPrediction= df2.dropna()

NoPrediction.describe()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Total Yards_Away_Team,Rolling 5 Past H&A Total Yards Avg_Away_Team,Rolling 5 Past H&A Yards per Pass_Away_Team,Rolling 5 Past H&A Yards per Pass Avg_Away_Team,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x
count,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,...,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000,929.000000
mean,-2.789020,2.789020,51.569429,127.890205,25.578041,98.277718,19.655544,1863.542519,372.708504,36.130248,...,1868.631862,373.726372,36.262217,7.252443,-2.134015,-0.426803,58.674919,57.356297,58.451023,57.642626
std,13.278237,13.278237,8.107559,42.611072,8.522214,17.719752,3.543950,378.310811,75.662162,7.395493,...,366.846878,73.369376,7.367379,1.473476,48.485785,9.697157,44.309013,44.575114,44.447301,44.472184
min,-41.500000,-36.000000,27.000000,33.000000,6.600000,47.000000,9.400000,957.000000,191.400000,18.000000,...,952.000000,190.400000,16.900000,3.380000,-148.500000,-29.700000,1.000000,1.000000,1.000000,1.000000
25%,-12.500000,-6.500000,45.500000,99.000000,19.800000,87.000000,17.400000,1601.000000,320.200000,31.200000,...,1645.000000,329.000000,31.100000,6.220000,-33.000000,-6.600000,11.000000,10.000000,10.000000,10.000000
50%,-3.000000,3.000000,50.500000,124.000000,24.800000,97.000000,19.400000,1861.000000,372.200000,35.700000,...,1855.000000,371.000000,35.900000,7.180000,-0.500000,-0.100000,99.000000,99.000000,99.000000,99.000000
75%,6.500000,12.500000,56.500000,152.000000,30.400000,110.000000,22.000000,2111.000000,422.200000,40.800000,...,2107.000000,421.400000,40.900000,8.180000,34.000000,6.800000,99.000000,99.000000,99.000000,99.000000
max,36.000000,41.500000,82.500000,261.000000,52.200000,155.000000,31.000000,2987.000000,597.400000,64.300000,...,3057.000000,611.400000,68.200000,13.640000,135.500000,27.100000,99.000000,99.000000,99.000000,99.000000


In [11]:
columns= NoPrediction.columns


In [12]:
NoPrediction.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Total Yards Avg_Away_Team,Rolling 5 Past H&A Yards per Pass_Away_Team,Rolling 5 Past H&A Yards per Pass Avg_Away_Team,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,H/A Winner
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,495.4,46.5,9.30,-87.5,-17.5,3.0,1.0,3,1,
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,438.8,50.5,10.10,-109.5,-21.9,1.0,3.0,3,1,Home
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,414.8,34.5,6.90,-18.5,-3.7,11.0,99.0,99,15,Home
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,523.6,50.9,10.18,-110.5,-22.1,1.0,99.0,99,3,Away
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,426.8,51.0,10.20,34.0,6.8,99.0,99.0,99,99,Away


In [13]:
NoPrediction = NoPrediction[1:]

In [14]:
NoPrediction.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Total Yards Avg_Away_Team,Rolling 5 Past H&A Yards per Pass_Away_Team,Rolling 5 Past H&A Yards per Pass Avg_Away_Team,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,H/A Winner
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,438.8,50.5,10.10,-109.5,-21.9,1.0,3.0,3,1,Home
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,414.8,34.5,6.90,-18.5,-3.7,11.0,99.0,99,15,Home
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,523.6,50.9,10.18,-110.5,-22.1,1.0,99.0,99,3,Away
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,426.8,51.0,10.20,34.0,6.8,99.0,99.0,99,99,Away
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,365.4,32.0,6.40,46.0,9.2,99.0,21.0,25,99,Home


In [15]:
#Feature Selection


features = []
for i in columns:
    features.append(i)
features.remove('H/A Winner')



X = NoPrediction[features] 

y = NoPrediction['H/A Winner']

# Model 1 - entr tree.DecisionTreeClassifier

In [16]:
# Call train_test_split on X, y. Make the test_size = 0.25, and random_state = 246
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=.25,random_state=246)

In [17]:
# Declare a variable called entr_model and use tree.DecisionTreeClassifier. 
entr_model= tree.DecisionTreeClassifier(criterion="entropy",random_state = 1234)
# Call fit() on entr_model
entr_model.fit(X_train,y_train)

# Call predict() on entr_model with X_test passed to it, and assign the result to a variable y_pred 
y_pred = entr_model.predict(X_test)

# Call Series on our y_pred variable with the following: pd.Series(y_pred)
y_pred = pd.Series(y_pred)

# Check out entr_model
entr_model

DecisionTreeClassifier(criterion='entropy', random_state=1234)

In [18]:
# As before, we need to visualize the tree to grasp its nature
#dot_data = StringIO()
#tree.export_graphviz(entr_model, out_file=dot_data,  
              #  filled=True, rounded=True,
              #  special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])

# Alternatively for class_names use entr_model2.classes_
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [19]:
# Run this block for model evaluation metrics 
print("Model Entropy - no max depth")
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
print('Precision score for "Home"' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
print('Precision score for "Away"' , metrics.precision_score(y_test,y_pred, pos_label = "Away"))
print('Recall score for "Home"' , metrics.recall_score(y_test,y_pred, pos_label = "Home"))
print('Recall score for "Away"' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

Model Entropy - no max depth
Accuracy: 0.6422413793103449
Balanced accuracy: 0.6473651960784315
Precision score for "Home" 0.7304347826086957
Precision score for "Away" 0.5555555555555556
Recall score for "Home" 0.6176470588235294
Recall score for "Away" 0.6770833333333334


# Model 2 gini tree.DecisionTreeClassifier

In [20]:
# Make a variable called gini_model, and assign it exactly what you assigned entr_model with above, but with the
# criterion changed to 'gini'
gini_model= tree.DecisionTreeClassifier(criterion="gini",random_state = 1234)
# Call fit() on the gini_model as you did with the entr_model
gini_model.fit(X_train,y_train)

# Call predict() on the gini_model as you did with the entr_model 
y_pred = gini_model.predict(X_test)

# Turn y_pred into a series, as before
y_pred = pd.Series(y_pred)

# Check out gini_model
gini_model

DecisionTreeClassifier(random_state=1234)

In [21]:
# As before, but make the model name gini_model

#dot_data = StringIO()
#tree.export_graphviz(gini_model, out_file=dot_data,  
               # filled=True, rounded=True,
               # special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])
# Alternatively for class_names use gini_model.classes_
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [22]:
# Run this block for model evaluation
print("Model Gini impurity model")
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

Model Gini impurity model
Accuracy: 0.6637931034482759
Balanced accuracy: 0.670343137254902
Precision score 0.7543859649122807
Recall score 0.7083333333333334


# Model 3 - entr tree.DecisionTreeClassifier w/ max depth 3

In [23]:
# Made a model as before, but call it entr_model2, and make the max_depth parameter equal to 3. 
# Execute the fitting, predicting, and Series operations as before
entr_model2 = tree.DecisionTreeClassifier(criterion="entropy",random_state = 1234,max_depth = 3)
entr_model2.fit(X_train,y_train)
y_pred = entr_model2.predict(X_test)
y_pred = pd.Series(y_pred)
entr_model2


DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=1234)

In [24]:
# As before, we need to visualize the tree to grasp its nature

#dot_data = StringIO()
#tree.export_graphviz(entr_model2, out_file=dot_data,  
               # filled=True, rounded=True,
               # special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])
# Alternatively for class_names use entr_model2.classes_

#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [25]:
# Run this block for model evaluation 
print("Model Entropy model max depth 3")
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
print('Precision score for "Home"' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
print('Recall score for "Away"' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

Model Entropy model max depth 3
Accuracy: 0.7112068965517241
Balanced accuracy: 0.7184436274509804
Precision score for "Home" 0.8
Recall score for "Away" 0.7604166666666666


# Model 4 - gini - tree.DecisionTreeClassifier Max depth 3

In [26]:
# As before, make a variable, but call it gini_model2, and ensure the max_depth parameter is set to 3
gini_model2 = tree.DecisionTreeClassifier(criterion ='gini', random_state = 1234, max_depth  = 3 )

# Do the fit, predict, and series transformations as before. 
gini_model2.fit(X_train,y_train)
y_pred = gini_model2.predict(X_test)
y_pred = pd.Series(y_pred)
gini_model2

DecisionTreeClassifier(max_depth=3, random_state=1234)

In [27]:
#dot_data = StringIO()
#tree.export_graphviz(gini_model2, out_file=dot_data,  
            #    filled=True, rounded=True,
             #   special_characters=True, feature_names=X_train.columns,class_names = ["Away", "Home"])
# Alternatively for class_names use gini_model2.classes_

#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#Image(graph.create_png())

In [28]:
print("Gini impurity  model - max depth 3")
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

Gini impurity  model - max depth 3
Accuracy: 0.7068965517241379
Balanced accuracy: 0.7132352941176471
Precision score 0.7931034482758621
Recall score 0.75


In [29]:
# Call value_counts() on the 'Decision' column of the original coffeeData
df2['H/A Winner'].value_counts()

Home    537
Away    439
          1
Name: H/A Winner, dtype: int64

In [30]:

feature_cols = features


new_X = Prediction[feature_cols]

In [31]:
# Call get_dummies() on the Pandas object pd, with new_X plugged in, to one-hot encode all features in the training set
new_X = pd.get_dummies(new_X)

# Make a variable called potential_buyers, and assign it the result of calling predict() on a model of your choice; 
# don't forget to pass new_X to predict()

Home_Wins = gini_model2.predict(new_X)

In [32]:
np.unique(Home_Wins, return_counts=True)

(array(['Away'], dtype=object), array([1]))

# Model 5 - RF /entr / max_depth =2

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

In [34]:
# Plug in appropriate max_depth and random_state parameters 
firstRFModel = RandomForestClassifier(max_depth= 2, random_state= 1234, criterion = 'entropy',)
firstRFModel.fit(X_train, y_train)


# Model and fit
y_pred = firstRFModel.predict(X_test)
y_pred = pd.Series(y_pred)


In [35]:
print("RF  model - max depth 3")
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

RF  model - max depth 3
Accuracy: 0.7456896551724138
Balanced accuracy: 0.7417279411764706
Precision score 0.7938931297709924
Recall score 0.71875


In [36]:
Prediction

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Total Yards Avg_Away_Team,Rolling 5 Past H&A Yards per Pass_Away_Team,Rolling 5 Past H&A Yards per Pass Avg_Away_Team,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,H/A Winner
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,495.4,46.5,9.3,-87.5,-17.5,3.0,1.0,3,1,


In [37]:
X = Prediction.drop('H/A Winner',axis =1)
X

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Total Yards_Away_Team,Rolling 5 Past H&A Total Yards Avg_Away_Team,Rolling 5 Past H&A Yards per Pass_Away_Team,Rolling 5 Past H&A Yards per Pass Avg_Away_Team,Rolling 5 Past H&A Spread_Away_Team,Rolling 5 Past H&A Spread Avg_Away_Team,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,2477.0,495.4,46.5,9.3,-87.5,-17.5,3.0,1.0,3,1


In [38]:
firstRFModel.predict(X) #Predicted Georgia to Win

array(['Home'], dtype=object)

In [39]:
firstRFModel.predict_proba(X) #Slight edge given to Georgia, with 53% chance of winning.

array([[0.46945465, 0.53054535]])

In [40]:
#Plug in appropriate max_depth and random_state parameters 
#secondRFModel = RandomForestClassifier(random_state= 1234)
#secondRFModel.fit(X_train, y_train)


 #Model and fit
#y_pred = secondRFModel.predict(X_test)
#y_pred = pd.Series(y_pred)


In [41]:
#print("RF2  model - max depth 3")
#print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
#print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
#print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
#print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

# Did manual calculations below to better understand the accuracy/precision/recall/balanced accuracy score

In [42]:
y_test.describe()

count      232
unique       2
top       Home
freq       136
Name: H/A Winner, dtype: object

In [43]:
y_pred.describe()

count      232
unique       2
top       Home
freq       131
dtype: object

In [44]:
y_test2 = y_test.reset_index(drop=True)

In [45]:
y_test2 

0      Home
1      Home
2      Home
3      Home
4      Home
       ... 
227    Home
228    Away
229    Away
230    Away
231    Home
Name: H/A Winner, Length: 232, dtype: object

In [46]:
y_pred

0      Home
1      Home
2      Away
3      Home
4      Home
       ... 
227    Home
228    Home
229    Away
230    Home
231    Home
Length: 232, dtype: object

In [47]:
y_pred_df = pd.DataFrame(y_pred)

In [48]:
y_test_df = pd.DataFrame(y_test2)

In [49]:
y_pred_df['y_test'] = y_test_df['H/A Winner']

In [50]:
y_test_df

,H/A Winner
0,Home
1,Home
2,Home
3,Home
4,Home
...,...
227,Home
228,Away
229,Away
230,Away


In [51]:
y_pred_df

,0,y_test
0,Home,Home
1,Home,Home
2,Away,Home
3,Home,Home
4,Home,Home
...,...,...
227,Home,Home
228,Home,Away
229,Away,Away
230,Home,Away


In [52]:
y_pred_df['Home Right'] = y_pred_df.apply(lambda x: 1 if (x['y_test'] == 'Home') & (x[0] == 'Home') else(0),axis=1)
y_pred_df['Home Wrong'] = y_pred_df.apply(lambda x: 1 if (x['y_test'] == 'Home') & (x[0] == 'Away') else(0),axis=1)
y_pred_df['Away Right'] = y_pred_df.apply(lambda x: 1 if (x['y_test'] == 'Away') & (x[0] == 'Away') else(0),axis=1)
y_pred_df['Away Wrong'] = y_pred_df.apply(lambda x: 1 if (x['y_test'] == 'Away') & (x[0] == 'Home') else(0),axis=1)

In [53]:
y_pred_df

,0,y_test,Home Right,Home Wrong,Away Right,Away Wrong
0,Home,Home,1,0,0,0
1,Home,Home,1,0,0,0
2,Away,Home,0,1,0,0
3,Home,Home,1,0,0,0
4,Home,Home,1,0,0,0
...,...,...,...,...,...,...
227,Home,Home,1,0,0,0
228,Home,Away,0,0,0,1
229,Away,Away,0,0,1,0
230,Home,Away,0,0,0,1


In [54]:
y_pred_df['Home Right'].sum() #Home Team Winning predicted correctly

104

In [55]:
y_pred_df['Home Wrong'].sum() # Home Team Winning predicted incorrectly

32

In [56]:
y_pred_df['Away Right'].sum() # Away Team Winning predicted correctly

69

In [57]:
y_pred_df['Away Wrong'].sum() # Away Team Winning predicted incorrectly

27

In [58]:
accuracy_count = y_pred_df['Away Right'].sum() + y_pred_df['Home Right'].sum() #Away & Home Team Winning predicted correctly 

In [59]:
total_count=y_pred_df['y_test'].count() #Total games

In [60]:
accuracy_count/total_count # correct predictions %

0.7456896551724138

In [61]:
H_correct = y_pred_df['Home Right'].sum() / (y_pred_df['Home Right'].sum() + y_pred_df['Home Wrong'].sum()) #Home Team correctly predicted percentage

In [62]:
y_pred_df['Away Right'].sum() / (y_pred_df['Away Right'].sum() + y_pred_df['Away Wrong'].sum()) #Recall score #Away Team correctly predicted percentage

0.71875

In [63]:
A_correct = y_pred_df['Away Right'].sum() / (y_pred_df['Away Right'].sum() + y_pred_df['Away Wrong'].sum()) #Away Team correctly predicted percentage

In [64]:
(y_pred_df['Away Right'].sum() + y_pred_df['Home Right'].sum())  / y_pred_df['y_test'].count() #Accuracy score

0.7456896551724138

In [65]:
(H_correct + A_correct) / 2 # Balanced Accuracy

0.7417279411764706

In [66]:
y_pred_df['Home Right'].sum() / (y_pred_df['Home Right'].sum()  + y_pred_df['Away Wrong'].sum()) #precision score
    
    #tp / (tp + fp)

0.7938931297709924

In [67]:
print("RF  model - max depth 3")
print("Accuracy:", metrics.accuracy_score(y_test,y_pred))
print("Balanced accuracy:", metrics.balanced_accuracy_score(y_test,y_pred))
print('Precision score' , metrics.precision_score(y_test,y_pred, pos_label = "Home"))
print('Recall score' , metrics.recall_score(y_test,y_pred, pos_label = "Away"))

RF  model - max depth 3
Accuracy: 0.7456896551724138
Balanced accuracy: 0.7417279411764706
Precision score 0.7938931297709924
Recall score 0.71875


# Did some calculations below to compare the models performance above vs simply using ranking or home team/ away team to make winning team prediction. 

In [68]:
df = pd.read_excel(r'/Users/markclampitt/Documents/Springboard/Capstone2/Data\new_df.xlsx')

In [69]:
df.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x
0,-2.5,2.5,52.0,172.0,34.4,119.0,23.8,2211.0,442.2,46.6,...,3.0,1.0,3,1,0.0,5.0,1.0,5.0,NaN,NaN
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,1.0,3.0,3,1,5.0,0.0,5.0,0.0,24.0,41.0
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,11.0,99.0,99,15,16.0,11.0,21.0,11.0,24.0,27.0
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,1.0,99.0,99,3,11.0,5.0,16.0,5.0,24.0,22.0
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99.0,99.0,99,99,25.0,13.0,35.0,18.0,45.0,21.0


In [70]:
df= df[1:]

In [71]:
df.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Away_Team,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,1.0,3.0,3,1,5.0,0.0,5.0,0.0,24.0,41.0
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,11.0,99.0,99,15,16.0,11.0,21.0,11.0,24.0,27.0
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,1.0,99.0,99,3,11.0,5.0,16.0,5.0,24.0,22.0
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99.0,99.0,99,99,25.0,13.0,35.0,18.0,45.0,21.0
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,99.0,21.0,25,99,15.0,18.0,15.0,23.0,17.0,34.0


In [72]:
df['H/A Winner'] = df.apply(lambda x: 'Home' if x['Home Score_x_x'] > x['Away Score_x_x'] else ('Away' if x['Away Score_x_x'] > x['Home Score_x_x'] else ""),axis=1)

In [73]:
df.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Previous H&A Ranking_Home_Team,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,3.0,3,1,5.0,0.0,5.0,0.0,24.0,41.0,Home
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,99.0,99,15,16.0,11.0,21.0,11.0,24.0,27.0,Home
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,99.0,99,3,11.0,5.0,16.0,5.0,24.0,22.0,Away
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99.0,99,99,25.0,13.0,35.0,18.0,45.0,21.0,Away
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,21.0,25,99,15.0,18.0,15.0,23.0,17.0,34.0,Home


In [74]:
df['H/A Winner'].unique()

array(['Home', 'Away'], dtype=object)

In [75]:
df['Ranked Game'] = df.apply(lambda x: 1 if ((x['Home Team Ranking_x'] < 99) or (x['Away Team Ranking_x'] < 99)) else (0),axis=1)



In [76]:
df.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Home Team Ranking_x,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner,Ranked Game
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,3,1,5.0,0.0,5.0,0.0,24.0,41.0,Home,1
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,99,15,16.0,11.0,21.0,11.0,24.0,27.0,Home,1
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,99,3,11.0,5.0,16.0,5.0,24.0,22.0,Away,1
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99,99,25.0,13.0,35.0,18.0,45.0,21.0,Away,0
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,25,99,15.0,18.0,15.0,23.0,17.0,34.0,Home,1


In [77]:
df['Ranked Game'].sum()

674

In [78]:
df.shape

(976, 39)

In [79]:
674/976 #70% games have at least one ranked team

0.6905737704918032

In [80]:
df['Both Teams Ranked Game'] = df.apply(lambda x: 1 if ((x['Home Team Ranking_x'] < 99) & (x['Away Team Ranking_x'] < 99)) else (0),axis=1)



In [81]:
df.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Away Team Ranking_x,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner,Ranked Game,Both Teams Ranked Game
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,1,5.0,0.0,5.0,0.0,24.0,41.0,Home,1,1
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,15,16.0,11.0,21.0,11.0,24.0,27.0,Home,1,0
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,3,11.0,5.0,16.0,5.0,24.0,22.0,Away,1,0
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,99,25.0,13.0,35.0,18.0,45.0,21.0,Away,0,0
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,99,15.0,18.0,15.0,23.0,17.0,34.0,Home,1,0


In [82]:
df['Both Teams Ranked Game'].sum()

220

In [83]:
220/976 # 22% of Games both teams are ranked

0.22540983606557377

In [84]:
#Ranked_Games_df= df[df['Ranked Game']==1]

In [85]:
#Ranked_Games_df.shape

In [86]:
df['Lower Ranked Team Wins'] = df.apply(lambda x: 1 if ((x['Home Team Ranking_x'] < x['Away Team Ranking_x']) & (x['Home Score_x_x'] > x['Away Score_x_x'])) or ((x['Away Team Ranking_x'] < x['Home Team Ranking_x']) & (x['Away Score_x_x'] > x['Home Score_x_x'])) else (0),axis=1)

df.sample(10)


,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Conf T Losses_Home_Team,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner,Ranked Game,Both Teams Ranked Game,Lower Ranked Team Wins
617,-10.5,10.5,59.0,139.0,27.8,95.0,19.0,1801.0,360.2,31.6,...,9.0,0.0,10.0,0.0,16.0,13.0,Away,1,0,0
583,6.0,-6.0,43.0,67.0,13.4,63.0,12.6,1177.0,235.4,23.3,...,22.0,24.0,27.0,24.0,7.0,10.0,Home,0,0,0
625,17.5,-17.5,44.0,131.0,26.2,112.0,22.4,2141.0,428.2,38.5,...,23.0,13.0,23.0,13.0,48.0,10.0,Away,1,0,1
692,-5.5,5.5,47.5,86.0,17.2,96.0,19.2,1789.0,357.8,31.2,...,9.0,11.0,9.0,16.0,17.0,42.0,Home,0,0,0
26,-1.0,1.0,47.0,156.0,31.2,133.0,26.6,2191.0,438.2,33.4,...,13.0,1.0,17.0,1.0,17.0,31.0,Home,1,0,0
382,-7.0,7.0,54.5,103.0,20.6,92.0,18.4,1770.0,354.0,36.9,...,15.0,12.0,21.0,12.0,46.0,54.0,Home,0,0,0
253,-4.5,4.5,40.5,124.0,24.8,92.0,18.4,1616.0,323.2,32.6,...,11.0,10.0,11.0,15.0,20.0,28.0,Home,0,0,0
70,6.0,-6.0,48.5,148.0,29.6,119.0,23.8,2181.0,436.2,39.7,...,11.0,5.0,11.0,5.0,31.0,20.0,Away,1,0,1
945,16.0,-16.0,47.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0.0,NaN,0.0,35.0,14.0,Away,1,0,1
870,-27.5,27.5,52.5,123.0,24.6,99.0,19.8,1594.0,318.8,30.1,...,9.0,19.0,9.0,28.0,7.0,26.0,Home,1,0,1


In [87]:
df['Lower Ranked Team Wins'].sum()

500

In [88]:
500/674 # of the games with at least one team ranked (674), 74% of the time the lower ranked team wins 

0.7418397626112759

In [89]:
#Both_Teams_Ranked_df = df[df['Both Teams Ranked Game'] ==1]

In [90]:
#Both_Teams_Ranked_df.shape

In [91]:
#Both_Teams_Ranked_df.head()

In [92]:
df['One Team Ranked'] = df.apply(lambda x: 1 if (x['Both Teams Ranked Game'] == 0) & (x['Ranked Game'] ==1) else(0),axis=1)

In [93]:
df.sample(10)

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner,Ranked Game,Both Teams Ranked Game,Lower Ranked Team Wins,One Team Ranked
96,17.0,-17.0,42.0,122.0,24.4,83.0,16.6,1539.0,307.8,28.5,...,2.0,10.0,3.0,14.0,3.0,Away,1,0,1,1
858,-14.5,14.5,39.5,140.0,28.0,90.0,18.0,1796.0,359.2,34.0,...,16.0,5.0,21.0,24.0,22.0,Away,1,0,0,1
276,-2.0,2.0,45.0,166.0,33.2,110.0,22.0,2308.0,461.6,42.6,...,10.0,13.0,10.0,15.0,9.0,Away,0,0,0,0
822,-3.5,3.5,46.0,82.0,16.4,91.0,18.2,1603.0,320.6,31.3,...,2.0,18.0,8.0,38.0,41.0,Home,1,0,0,1
436,-18.0,18.0,63.0,202.0,40.4,129.0,25.8,2672.0,534.4,48.1,...,8.0,3.0,8.0,35.0,42.0,Home,1,0,1,1
369,-21.0,21.0,38.5,142.0,28.4,88.0,17.6,1823.0,364.6,38.9,...,17.0,2.0,22.0,7.0,9.0,Home,1,0,1,1
255,14.0,-14.0,49.5,130.0,26.0,102.0,20.4,1906.0,381.2,27.7,...,0.0,7.0,0.0,31.0,24.0,Away,1,1,1,0
429,-1.0,1.0,54.0,140.0,28.0,100.0,20.0,1908.0,381.6,38.9,...,3.0,1.0,3.0,35.0,31.0,Away,1,1,1,0
810,-3.0,3.0,51.5,149.0,29.8,92.0,18.4,1872.0,374.4,28.7,...,7.0,9.0,10.0,9.0,7.0,Away,1,0,1,1
295,-5.5,5.5,47.0,107.0,21.4,89.0,17.8,1686.0,337.2,38.3,...,13.0,17.0,18.0,23.0,13.0,Away,0,0,0,0


In [94]:
One_Team_Ranked_df = df[df['One Team Ranked'] ==1]

In [95]:
Both_Teams_Ranked_df = df[df['Both Teams Ranked Game'] ==1]

In [96]:
One_Team_Ranked_df.shape

(454, 42)

In [97]:
One_Team_Ranked_df['Lower Ranked Team Wins'].sum()

369

In [98]:
369/454

0.8127753303964758

In [99]:
# when there is only one team ranked 80% of the time the ranked team wins. 

In [100]:
Both_Teams_Ranked_df.shape

(220, 42)

In [101]:
Both_Teams_Ranked_df['Lower Ranked Team Wins'].sum()

131

In [102]:
131/220

0.5954545454545455

In [103]:
#When both teams are ranked - 59% of the time the lower ranked team wins. 

In [104]:
df.shape

(976, 42)

In [105]:
df['Lower Ranked Team Wins'].sum()

500

In [106]:
(131+369)/(454+220)

0.7418397626112759

In [107]:
#Overall 70% of the games where at least one team is ranked, 74% of the time the lower ranked team wins.

In [108]:
df.head()

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Conf T Losses_Away_Team,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner,Ranked Game,Both Teams Ranked Game,Lower Ranked Team Wins,One Team Ranked
1,6.5,-6.5,49.0,187.0,37.4,118.0,23.6,2484.0,496.8,49.3,...,0.0,5.0,0.0,24.0,41.0,Home,1,1,0,0
2,6.5,-6.5,27.0,114.0,22.8,100.0,20.0,1791.0,358.2,30.9,...,11.0,21.0,11.0,24.0,27.0,Home,1,0,0,1
3,19.5,-19.5,55.5,123.0,24.6,111.0,22.2,1998.0,399.6,38.5,...,5.0,16.0,5.0,24.0,22.0,Away,1,0,1,1
4,31.5,-31.5,63.5,88.0,17.6,81.0,16.2,1569.0,313.8,31.4,...,13.0,35.0,18.0,45.0,21.0,Away,0,0,0,0
5,-14.5,14.5,63.0,156.0,31.2,125.0,25.0,2280.0,456.0,40.4,...,18.0,15.0,23.0,17.0,34.0,Home,1,0,1,1


In [109]:
df[df['H/A Winner'] == 'Home']['H/A Winner'].count()

537

In [110]:
df[df['H/A Winner'] == 'Away']['H/A Winner'].count()

439

In [111]:
439+537

976

In [112]:
537/976

0.5502049180327869

In [113]:
#55% of the time the home team wins

In [114]:
df['Lower Spread Wins'] = df.apply(lambda x: 1 if ((x['Home Team Spread_x'] < x['Away Team Spread_x']) & (x['Home Score_x_x'] > x['Away Score_x_x'])) or ((x['Away Team Spread_x'] < x['Home Team Spread_x']) & (x['Away Score_x_x'] > x['Home Score_x_x'])) else (0),axis=1)

df.sample(10)

,Home Team Spread_x,Away Team Spread_x,Total_x,Rolling 5 Past H&A Score_Home_Team,Rolling 5 Past H&A Score Avg_Home_Team,Rolling 5 Past H&A 1st Downs_Home_Team,Rolling 5 Past H&A 1st Downs Avg_Home_Team,Rolling 5 Past H&A Total Yards_Home_Team,Rolling 5 Past H&A Total Yards Avg_Home_Team,Rolling 5 Past H&A Yards per Pass_Home_Team,...,Rolling 5 Past H&A Season T Losses_Home_Team,Rolling 5 Past H&A Season T Losses_Away_Team,Away Score_x_x,Home Score_x_x,H/A Winner,Ranked Game,Both Teams Ranked Game,Lower Ranked Team Wins,One Team Ranked,Lower Spread Wins
387,5.0,-5.0,54.5,143.0,28.6,92.0,18.4,1961.0,392.2,42.2,...,12.0,5.0,41.0,23.0,Away,1,1,0,0,1
239,8.0,-8.0,54.0,135.0,27.0,102.0,20.4,1806.0,361.2,32.7,...,12.0,5.0,41.0,17.0,Away,1,1,1,0,1
524,-14.0,14.0,61.0,145.0,29.0,103.0,20.6,2003.0,400.6,33.1,...,25.0,35.0,17.0,37.0,Home,0,0,0,0,1
49,-2.5,2.5,66.0,165.0,33.0,126.0,25.2,2196.0,439.2,34.6,...,16.0,26.0,62.0,24.0,Away,0,0,0,0,0
309,-3.0,3.0,53.5,90.0,18.0,90.0,18.0,1950.0,390.0,34.1,...,22.0,17.0,17.0,26.0,Home,0,0,0,0,1
584,-11.5,11.5,53.5,185.0,37.0,103.0,20.6,1926.0,385.2,50.7,...,0.0,5.0,17.0,41.0,Home,1,1,1,0,1
528,-18.0,18.0,48.5,107.0,21.4,92.0,18.4,1749.0,349.8,32.8,...,7.0,18.0,35.0,41.0,Home,1,0,1,1,1
383,-13.5,13.5,59.0,121.0,24.2,118.0,23.6,2057.0,411.4,35.4,...,7.0,10.0,16.0,42.0,Home,0,0,0,0,1
263,3.5,-3.5,57.0,129.0,25.8,78.0,15.6,1781.0,356.2,36.7,...,12.0,12.0,35.0,14.0,Away,0,0,0,0,1
794,-2.0,2.0,44.5,147.0,29.4,97.0,19.4,1873.0,374.6,36.4,...,8.0,12.0,20.0,45.0,Home,1,1,1,0,1


In [115]:
df['Lower Spread Wins'].sum()

739

In [116]:
df['Lower Spread Wins'].count()

976

In [117]:
739/976 #75% of the time the team with the lower spread (favored) wins the game 

0.757172131147541